# dvc_yaml

> Generate dvc.yaml file

In [71]:
pairs = ["BTCUSDT", "ETHUSDT", "ADAUSDT"]
output = "../dvc.yaml"

In [72]:
import os
import pprint

import yaml

In [73]:
os.getcwd()

'/home/mvkvc/dev/rlmm/nbs'

In [74]:
# What does this do again


def str_representer(dumper, data):
    if len(data.splitlines()) > 1:
        return dumper.represent_scalar(
            tag="tag:yaml.org,2002:str", value=data, style="|"
        )
    return dumper.represent_scalar(tag="tag:yaml.org,2002:str", value=data)

In [75]:
yaml.add_representer(str, str_representer)

In [76]:
test_cmd = ["touch data/.dummy_cache"]
test_deps = ["data/test"]
test_outs = ["data/.dummy_cache"]

In [77]:
csv_cmd = []
csv_deps = test_outs.copy()
csv_outs = []

for pair in pairs:
    csv_cmd += [
        f"mkdir -p data/csv/book_snapshot_25/{pair}/",
        f"mkdir -p data/csv/trades/{pair}/",
        f"cp -r data/raw/book_snapshot_25/{pair}/* data/csv/book_snapshot_25/{pair}/",
        f"cp -r data/raw/trades/{pair}/* data/csv/trades/{pair}/",
        f"gzip -d data/csv/book_snapshot_25/{pair}/*.csv.gz",
        f"gzip -d data/csv/trades/{pair}/*.csv.gz",
    ]
    csv_deps += [f"data/raw/book_snapshot_25/{pair}/", f"data/raw/trades/{pair}/"]
    csv_outs += [
        {f"data/csv/book_snapshot_25/{pair}/": {"cache": False, "persist": True}},
        {f"data/csv/trades/{pair}/": {"cache": False, "persist": True}},
    ]

In [78]:
parquet_cmd = []
parquet_deps = []
parquet_outs = []

for pair in pairs:
    parquet_cmd += [
        #         Change this to the new papermill format
        #         f"python src/dvc/stage_parquet.py --input data/csv/book_snapshot_25/{pair}/ --output data/parquet/book_snapshot_25/{pair}/ --time_column timestamp --time_rename ts -d exchange symbol local_timestamp",
        #         f"python src/dvc/stage_parquet.py --input data/csv/trades/{pair}/ --output data/parquet/trades/{pair}/ --time_column timestamp --time_rename ts -d exchange symbol local_timestamp id",
        f"papermill --cwd nbs/stage_parquet.py -p path_input data/csv/book_snapshot_25/{pair}/ -p output_path data/parquet/book_snapshot_25/{pair}/ -p column_time timestamp -p rename_time ts -p drop 'exchange,symbol,local_timestamp'"
        f"papermill --cwd nbs/stage_parquet.py -p path_input data/csv/trades/{pair}/ -p output_path data/parquet/trades/{pair}/ -p column_time timestamp -p rename_time ts -p drop 'exchange,symbol,local_timestamp,id'"
    ]
    parquet_deps += [
        f"data/csv/book_snapshot_25/{pair}/",
        f"data/csv/trades/{pair}/",
    ]
    parquet_outs += [
        f"data/parquet/book_snapshot_25/{pair}/",
        f"data/parquet/trades/{pair}/",
    ]

In [79]:
# TODO: Add constraints and/or more preprocessing when needed

# constraints_cmd = []
# constraints_deps = [x for x in parquet_outs if "book_snapshot_25" in x]
# constraints_outs = []

# for pair in pairs:
#     constraints_cmd += [
#         f"python src/dvc/stage_constraints.py data/parquet/book_snapshot_25/{pair}/ data/process/constraints/{pair}/"
#     ]
#     constraints_outs += [f"data/constraints/{pair}/"]

In [80]:
train_cmd = ["papermill --cwd nbs nbs/train.ipynb nbs/train.ipynb"]
train_deps = parquet_outs
train_outs = ["outs/"]

In [81]:
config = {
    "stages": {
        "test": {"cmd": test_cmd, "deps": test_deps, "outs": test_outs},
        "csv": {"cmd": csv_cmd, "deps": csv_deps, "outs": csv_outs},
        "parquet": {"cmd": parquet_cmd, "deps": parquet_deps, "outs": parquet_outs},
        "train": {
            "cmd": train_cmd,
            "deps": train_deps,
            "outs": train_outs,
        },
    }
}

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config)

{   'stages': {   'csv': {   'cmd': [   'mkdir -p '
                                        'data/csv/book_snapshot_25/BTCUSDT/',
                                        'mkdir -p data/csv/trades/BTCUSDT/',
                                        'cp -r '
                                        'data/raw/book_snapshot_25/BTCUSDT/* '
                                        'data/csv/book_snapshot_25/BTCUSDT/',
                                        'cp -r data/raw/trades/BTCUSDT/* '
                                        'data/csv/trades/BTCUSDT/',
                                        'gzip -d '
                                        'data/csv/book_snapshot_25/BTCUSDT/*.csv.gz',
                                        'gzip -d '
                                        'data/csv/trades/BTCUSDT/*.csv.gz',
                                        'mkdir -p '
                                        'data/csv/book_snapshot_25/ETHUSDT/',
                                        'mkdir -p 

In [82]:
output = os.path.abspath(output)

print(output)

/home/mvkvc/dev/rlmm/dvc.yaml


In [83]:
with open(f"{output}", "w", encoding="utf-8") as file:
    yaml.dump(data=config, stream=file, sort_keys=False)

In [84]:
!cat {output}

stages:
  test:
    cmd:
    - touch data/.dummy_cache
    deps:
    - data/test
    outs:
    - data/.dummy_cache
  csv:
    cmd:
    - mkdir -p data/csv/book_snapshot_25/BTCUSDT/
    - mkdir -p data/csv/trades/BTCUSDT/
    - cp -r data/raw/book_snapshot_25/BTCUSDT/* data/csv/book_snapshot_25/BTCUSDT/
    - cp -r data/raw/trades/BTCUSDT/* data/csv/trades/BTCUSDT/
    - gzip -d data/csv/book_snapshot_25/BTCUSDT/*.csv.gz
    - gzip -d data/csv/trades/BTCUSDT/*.csv.gz
    - mkdir -p data/csv/book_snapshot_25/ETHUSDT/
    - mkdir -p data/csv/trades/ETHUSDT/
    - cp -r data/raw/book_snapshot_25/ETHUSDT/* data/csv/book_snapshot_25/ETHUSDT/
    - cp -r data/raw/trades/ETHUSDT/* data/csv/trades/ETHUSDT/
    - gzip -d data/csv/book_snapshot_25/ETHUSDT/*.csv.gz
    - gzip -d data/csv/trades/ETHUSDT/*.csv.gz
    - mkdir -p data/csv/book_snapshot_25/ADAUSDT/
    - mkdir -p data/csv/trades/ADAUSDT/
    - cp -r data/raw/book_snapshot_25/ADAUSDT/* data/csv/book_snapshot_25/ADAUSDT/
    - cp -r data